# **Extracting Semantic Model Memory Size from the Fabric Capacity Metrics App for a Single Capacity**

# Check out this [blog post](https://bits2bi.com/2025/03/15/extracting-semantic-model-size-from-the-fabric-capacity-metrics-app) for details about this notebook

⚠️ Note
The DAX query to extract the semantic model size mentioned in this Notebook was written and tested against the Fabric Capacity Metrics App version 1031 (updated on September 11, 2025). It may not work with older or newer versions, as the semantic model structure and object names are subject to change.

In [ ]:
# Install the latest version of semantic link

## ........ UNCOMMENT THE FOLLOWING LINE IF YOU'RE RUNNING SPARK 3.3 OR BELOW OR TO UPDATE THE SEMANTIC LINK TO THE LATEST VERSION ........ ##

#%pip install -U semantic-link

In [ ]:
# Import the Semantic Link Python library
import sempy.fabric as fabric

# Import other libraries and modules
from pyspark.sql.functions import lit, current_timestamp, to_date, col

In [ ]:
# Declare and assign variables

## ........ PROVIDE THE NAME OR ID OF YOUR FABRIC CAPACTIY METRICS WORKSPACE, FABRIC CAPACTIY METRICS SEMANTIC MODEL, AND THE CAPACITY ID IN FOLLOWING VARIABLES ........ ##


workspace_Name = "212091bd-0616-4370-91e8-c2b2fcf48b25"
dataset_Name = "dc9b689e-fb5a-4261-8e09-ffbb80091ce9"
capacity_Id = "d16efc02-0391-43bd-8841-47a044dd67ce"

In [ ]:
# Construct DAX query
dax_query = f"""
// DAX query to return the memory usage statistics for the previous day
// for all the semantic models hosted on a capacity
DEFINE
  // Provide a capacity ID for the MPARAMETER
    MPARAMETER 'CapacitiesList' = "{capacity_Id}"
// Filter the items for Datasets as we only want the memory information for semantic models
    VAR __DS0FilterTable =
        TREATAS ( {{ "Dataset" }}, 'Items'[Item kind] )
// Filter for yesterday as today's statistics are not yet final and may change during the day
// Always extract the data for the completed days
    VAR __DS0FilterTable2 =
        TREATAS ( {{ UTCTODAY () - 1 }}, 'Dates'[Day] )
// Extract Minimum, Maximum, and Median memory sizes
    VAR __DS0Core =
        SUMMARIZECOLUMNS (
            'Dates'[Day],
            Items[Item Id],
            Items[Item name],
            __DS0FilterTable,
            __DS0FilterTable2,
            "MinimumMemoryInGB", ROUND ( MIN ( 'Max Memory By Item And Hour'[Item size (GB)] ), 4 ),
            "MaximumMemoryInGB", ROUND ( MAX ( 'Max Memory By Item And Hour'[Item size (GB)] ), 4 ),
            "MedianMemoryInGB", ROUND ( MEDIAN ( 'Max Memory By Item And Hour'[Item size (GB)] ), 4 )
        )

EVALUATE
__DS0Core
"""

In [ ]:
# Execute DAX query on the semantic model
# https://learn.microsoft.com/en-us/python/api/semantic-link-sempy/sempy.fabric?view=semantic-link-python#sempy-fabric-evaluate-dax

dax_result = fabric.evaluate_dax(
    workspace = workspace_Name, 
    dataset = dataset_Name,
    dax_string = dax_query
)

display(dax_result)

In [ ]:
# Convert the DAX result to spark dataframe
dax_result_df = spark.createDataFrame(dax_result)

# Add the current time as data extraction date
dax_result_df = dax_result_df.withColumn("ExtractionDate", lit(current_timestamp()))

dax_result_df.printSchema()

In [ ]:
# Rename the columns for clarity
dax_result_cleaned_df = dax_result_df.withColumnRenamed("Dates[Day]", "Date") \
               .withColumn("Date", to_date(col("Date")))\
               .withColumnRenamed("Items[Item Id]", "SemanticModelID") \
               .withColumnRenamed("Items[Item Name]", "SemanticModelName") \
               .withColumnRenamed("[MinimumMemoryInGB]", "MinimumMemoryInGB") \
               .withColumnRenamed("[MaximumMemoryInGB]", "MaximumMemoryInGB") \
               .withColumnRenamed("[MedianMemoryInGB]", "MedianMemoryInGB")

# Total rows
print(f"Total Rows: {dax_result_cleaned_df.count()}")

dax_result_cleaned_df.printSchema()

# Show the data
display(dax_result_cleaned_df)

In [ ]:
# Append the data to the lakehouse delta table
dax_result_cleaned_df.write.format("delta").mode("append").saveAsTable("semantic_model_size_metrics")